<a href="https://colab.research.google.com/github/SeeraKhaled/Hackathon/blob/main/Mahroos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# Install required packages
!pip install fastapi uvicorn nest_asyncio pyngrok tldextract python-multipart
!pip install fuzzywuzzy[speedup]

In [22]:
from pyngrok import ngrok, conf
conf.get_default().auth_token = "36cIZprYapWhNJabGIRUiDs9iUQ_2WDNBmNkpVNeVMx69GAge"

### **Last copy of the code (works Perfectly)**

In [23]:
# Imports
from fastapi import FastAPI
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import os, json, tldextract
import asyncio
import datetime
import re
from fuzzywuzzy import fuzz

def log_attempt(input_type, value, status):
    os.makedirs("data", exist_ok=True)
    timestamp = datetime.datetime.now().isoformat(sep=" ", timespec="microseconds")

    # Unified log format: timestamp | type | value | status
    line = f"{timestamp} | {input_type} | {value} | {status}\n"

    # Log all attempts
    with open("data/logs.txt", "a", encoding="utf-8") as f:
        f.write(line)

    # Log suspicious attempts separately
    if status.lower() == "suspicious":
        with open("data/suspicious_logs.txt", "a", encoding="utf-8") as f:
            f.write(line)

# -----------------------------
# Step 1: Create whitelist/blacklist JSON files
# -----------------------------
os.makedirs("data", exist_ok=True)

whitelist = {
    "domains": ["absher.sa", "gov.sa", "my.gov.sa", "tawakkalna.sa"],
    "emails": ["noreply@absher.sa", "support@tawakkalna.sa", "info@sdaia.gov.sa", "Training@sar.com.sa"],
    "sms_senders": ["ABSHE R", "MOI", "TAWAKKALNA"],
    "phone_prefixes": ["+9669200", "+966800", "+966199"]
}

blacklist = {
    "domains": ["gov-sa-login.com", "abshr-secure.net", "sda1a.gov.sa", "tawakkalna.net"],
    "keywords": ["urgent", "verify account", "click here", "password", "reset"],
    "phones": ["+966512345678", "+966599999999"],
    "sms_samples": [
        "Urgent: verify your account now",
        "Click here to reset your password",
        "Your Absher account is locked, login now"
    ]
}

with open("data/whitelist.json","w", encoding="utf-8") as f:
    json.dump(whitelist,f,indent=2, ensure_ascii=False)

with open("data/blacklist.json","w", encoding="utf-8") as f:
    json.dump(blacklist,f,indent=2, ensure_ascii=False)

# -----------------------------
# Step 2: FastAPI app setup
# -----------------------------
app = FastAPI()

# Load whitelist/blacklist
with open("data/whitelist.json") as f:
    whitelist = json.load(f)
with open("data/blacklist.json") as f:
    blacklist = json.load(f)

@app.get("/")
def read_root():
    return {"message": "Hello from FastAPI in Colab!"}

# -----------------------------
# Step 3: Verification Endpoints
# -----------------------------

@app.post("/verify/email")
def verify_email(email: str):
    email = email.strip().lower()   # normalize input
    domain = email.split("@")[-1]

    # Whitelist check
    if email in [e.lower() for e in whitelist["emails"]]:
        result = {"status": "official", "detail": "Email is in whitelist"}
    elif domain in [d.lower() for d in whitelist["domains"]]:
        result = {"status": "official", "detail": "Domain is whitelisted"}
    elif domain in [d.lower() for d in blacklist["domains"]]:
        result = {"status": "suspicious", "detail": "Domain is blacklisted"}
    elif email in [e.lower() for e in blacklist.get("emails", [])]:
        result = {"status": "suspicious", "detail": "Email is blacklisted"}
    else:
        # Fuzzy domain check
        for bad_domain in blacklist["domains"]:
            score = fuzz.ratio(domain, bad_domain.lower())
            if score > 85:
                result = {"status": "suspicious", "detail": f"Domain similar to blacklisted: {bad_domain}"}
                break
        else:
            result = {"status": "unknown", "detail": "Not found in lists"}

    # Always log before returning
    log_attempt("email", email, result["status"])
    return result

@app.post("/verify/url")
def verify_url(url: str):
    domain = tldextract.extract(url).registered_domain

    if domain in whitelist["domains"]:
        result = {"status": "official", "detail": "Domain is whitelisted"}
    elif domain in blacklist["domains"]:
        result = {"status": "suspicious", "detail": "Domain is blacklisted"}
    else:
        # Fuzzy domain check
        for bad_domain in blacklist["domains"]:
            score = fuzz.ratio(domain, bad_domain)
            if score > 85:
                result = {"status": "suspicious", "detail": f"Domain similar to blacklisted: {bad_domain}"}
                break
        else:
            result = {"status": "unknown", "detail": "Not found in lists"}

    log_attempt("url", url, result["status"])
    return result

@app.post("/verify/phone")
def verify_phone(phone: str):
    phone = phone.strip()
    if not phone.startswith("+"):
        phone = "+" + phone

    for prefix in whitelist["phone_prefixes"]:
        if phone.startswith(prefix):
            result = {"status": "official", "detail": f"Phone prefix {prefix} is whitelisted"}
            log_attempt("phone", phone, result["status"])
            return result

    for bad in blacklist["phones"]:
        if phone == bad.strip():
            result = {"status": "suspicious", "detail": "Phone number is blacklisted"}
            log_attempt("phone", phone, result["status"])
            return result

    result = {"status": "unknown", "detail": "Not found in lists"}
    log_attempt("phone", phone, result["status"])
    return result

@app.post("/verify/sms")
def verify_sms(message: str):
    message_lower = message.lower()

    scam_patterns = {
        r"urgent": "Contains 'urgent'",
        r"verify\s+account": "Contains 'verify account'",
        r"click\s+here": "Contains 'click here'",
        r"reset\s+password": "Contains 'reset password'",
        r"account\s+locked": "Contains 'account locked'"
    }

    for pattern, description in scam_patterns.items():
        if re.search(pattern, message_lower):
            result = {"status": "suspicious", "detail": description}
            log_attempt("sms", message, result["status"])
            return result

    # Fuzzy match against scam samples
    for sample in blacklist["sms_samples"]:
        score = fuzz.partial_ratio(message_lower, sample.lower())
        if score > 80:
            result = {"status": "suspicious", "detail": f"Similar to known scam SMS (match {score}%)"}
            log_attempt("sms", message, result["status"])
            return result

    if any(sender in message for sender in whitelist["sms_senders"]):
        result = {"status": "official", "detail": "Sender looks whitelisted"}
    else:
        result = {"status": "unknown", "detail": "Not found in lists"}

    log_attempt("sms", message, result["status"])
    return result


# -----------------------------
# Step 4: Run with ngrok
# -----------------------------
nest_asyncio.apply()
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
server = uvicorn.Server(config)
await server.serve()

Public URL: NgrokTunnel: "https://monopetalous-moon-shrubbiest.ngrok-free.dev" -> "http://localhost:8000"


INFO:     Started server process [541]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     188.55.89.27:0 - "POST /verify/sms?message=account%20is%20locked,%20login%20now HTTP/1.1" 200 OK
INFO:     188.55.89.27:0 - "POST /verify/email?email=support@sda1a.gov.sa%0A HTTP/1.1" 200 OK
INFO:     188.55.89.27:0 - "POST /verify/phone?phone=+966512345678 HTTP/1.1" 200 OK
INFO:     188.55.89.27:0 - "POST /verify/url?url=https://tawakkalna.net HTTP/1.1" 200 OK


/tmp/ipython-input-2168112048.py:106: DeprecationWarning: The 'registered_domain' property is deprecated and will be removed in the next major version. Use 'top_domain_under_public_suffix' instead, which has the same behavior but a more accurate name.
  domain = tldextract.extract(url).registered_domain
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [541]


In [24]:
!cat data/logs.txt

2025-12-11 19:54:30.964921 | sms | click here | suspicious
2025-12-11 19:54:39.295781 | email | support@tawakkalna.sa | official
2025-12-11 20:05:15.012939 | email | support@tawakkalna.sa | official
2025-12-11 20:05:21.531774 | phone | +966599999999 | suspicious
2025-12-11 20:05:31.032496 | sms | click here | suspicious
2025-12-11 20:40:49.029680 | sms | account is locked, login now | suspicious
2025-12-11 20:41:24.448515 | email | support@sda1a.gov.sa | suspicious
2025-12-11 20:42:03.712923 | phone | +966512345678 | suspicious
2025-12-11 20:42:36.362227 | url | https://tawakkalna.net | suspicious


In [25]:
!cat data/suspicious_logs.txt

2025-12-11 19:54:30.965097 | sms | click here | suspicious
2025-12-11 20:05:21.531774 | phone | +966599999999 | suspicious
2025-12-11 20:05:31.032496 | sms | click here | suspicious
2025-12-11 20:40:49.029680 | sms | account is locked, login now | suspicious
2025-12-11 20:41:24.448515 | email | support@sda1a.gov.sa | suspicious
2025-12-11 20:42:03.712923 | phone | +966512345678 | suspicious
2025-12-11 20:42:36.362227 | url | https://tawakkalna.net | suspicious
